The following illustrates the problem with the shape of the result of the left orthogonalization. The input checking has been disabled so that the error is thrown from the algorithm itself.

In [1]:
using LinearAlgebra

In [2]:
# strong Kronecker product for 3rd order tensors
function strongKron(A::AbstractArray{<:Number, 3}, B::AbstractArray{<:Number, 3})
    m = size(A)
    n = size(B)
    @assert m[3] == n[1] "Shared rank index does not match: got $(m[3]) and $(n[1]) respectively."

    C = reshape(A, m[1]*m[2], m[3]) * reshape(B, n[1], n[2]*n[3])
    return reshape(C, m[1], m[2]*n[2], n[3])
end

⨝(A::AbstractArray{<:Number, 3}, B::AbstractArray{<:Number, 3}) = strongKron(A, B)

;

In [3]:
# MPS-TT type as a list of 3D-tensor factors
MPSTT{T<:Number} = AbstractVector{<:AbstractArray{T,3}}

# returns number of dimensions of an MPS-TT
dims(U::MPSTT) = length(U)

# returns the mode dimensions of an MPS-TT
function shape(U::MPSTT)
    n = [size(factor, 2) for factor in U]
    return Tuple(n)
end

# returns the ranks of an MPS-TT
function ranks(U::MPSTT)
    r = [size(factor, 1) for factor in U]
    r = r[2:end] # remove dummy dimension
    return Tuple(r)
end

# construct tensor from MPS-TT factors
function evalMPSTT(U::MPSTT)
    n = shape(U)
    A = reduce(strongKron, U)
    return reshape(A, n)
end

;

In [52]:
function roundMPSTT(U::MPSTT, p::NTuple{N,<:Integer}) where N

    # initialization
    d = dims(U)
    n = shape(U)
    r = (1, ranks(U)..., 1)
    p = (1, p..., 1)
    V = []
    singular_vals = Vector{Vector{Float64}}(undef, d-1)
    norms = Vector{Float64}(undef, d-1)

    # check input
    # @assert N == d-1 "Expected $(d-1) rank(s), got $N."
    # for k ∈ 1:d-1
    #     upper = min(p[k] * n[k], prod(n[k+1:end]))
    #     upper = min(upper, r[k+1])
    #     @assert 0 < p[k+1] <= upper "Rank at index $k is out of bounds: expected in range 1:$upper, got $(p[k+1])."
    # end

    # left orthogonalization
    R = ones(1, 1, 1)
    for k ∈ d:-1:2
        @show k
        UR = U[k] ⨝ R

        UR = permutedims(UR, [3,2,1]) # permute dims for right QR
        UR = reshape(UR, r[k+1]*n[k], r[k])

        Q, R = qr(UR)
        @show size(Q) size(R) size(Matrix(Q))

        Q = reshape(Matrix(Q), r[k+1], n[k], r[k])
        R = reshape(R, r[k], 1, r[k])

        Q = permutedims(Q, [3,2,1])
        R = permutedims(R, [3,2,1])

        push!(V, Q)
    end

    # compute first factor
    push!(V, U[1] ⨝ R)
    reverse!(V)

    # right approximation
    S = ones(1, 1, 1)
    for k ∈ 1:d-1
        SV = S ⨝ V[k]
        SV = reshape(SV, p[k]*n[k], r[k+1])

        F = svd(SV)
        Vk = F.U[:,1:p[k+1]]
        S = F.Vt[1:p[k+1],:] .* F.S[1:p[k+1]]

        singular_vals[k] = F.S
        norms[k] = norm(SV - Vk * S)

        Vk = reshape(Vk, p[k], n[k], p[k+1])
        S = reshape(S, p[k+1], 1, r[k+1])

        V[k] = Vk
    end

    # compute last factor
    V[d] = S ⨝ V[d]

    # bring all factors to the same type
    T = Base.promote_eltype(V...)
    V = [convert(Array{T}, factor) for factor ∈ V]

    return V, singular_vals, norms
end
;

In [53]:
function addMPSTT(A::MPSTT, B::MPSTT)
    d = dims(A)
    @assert d == dims(B) "Number of dimensions do not match: Got $d and $(dims(B))."
    @assert shape(A) == shape(B) "Mode dimensions do not match: Got $(shape(A)) and $(shape(B))."

    first_factor = cat(A[1], B[1], dims=3)
    middle_factors = [cat(A[k], B[k], dims=(1,3)) for k ∈ 2:d-1]
    last_factor = cat(A[d], B[d], dims=1)

    return [first_factor, middle_factors..., last_factor]
end
;

In [54]:
n = (10, 11, 12, 13)
rank = (n[1], n[2]*n[3], n[4])
r = [1, rank..., 1]
A = [rand(r[k], n[k], r[k+1]) for k ∈ 1:d]
B = [rand(r[k], n[k], r[k+1]) for k ∈ 1:d]
S = addMPSTT(A, B)
;

Maybe there is a problem in my MPS-TT sum implementation as well since the error is very big.

In [61]:
@assert shape(S) == n
@assert ranks(S) == 2 .* rank

norm(evalMPSTT(S) - evalMPSTT(A) + evalMPSTT(B))

241279.78891256524

In [56]:
roundMPSTT(S, ranks(S))

k = 4
size(Q) = (13, 13)
size(R) = (13, 26)
size(Matrix(Q)) = (13, 13)


DimensionMismatch: DimensionMismatch: new dimensions (1, 13, 26) must be consistent with array size 169

Error is thrown because one wants to reshape Q into the shape of the last factor of S, which is (1, 13, 26) but Q is just (13, 13).

Also the "trick" with multiplying Q with Matrix(I, size(R,2), size(R,2)) does not work because of this.

(The problem stems from the rank dimensions of the given MPS-TT factors S and not the given ranks for the rounding.)